In [1]:
#default_exp data.dataset

# Dataset dataloader
> In this file we will define the different dataloaders for out dataset. Primarily for the Kitti lidar dataset.py

## 00 - Prerequesits

In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")

### 00.1 Imports

In [3]:
#export
import logging
import os
import torch
import numpy as np
import random

"""from utils.math.compute_pillars import calculate_pillars"""
from utils.math.preprocessing import Preprocessing
from utils.io import read_config

logger = logging.getLogger(__name__)

## 01 - Dataset

In [4]:
#export
class VelTrainDataset(torch.utils.data.Dataset):
    """A dataset for loading Kitti velodyne dataset and calculating the pillar points and pillar indices"""

    def __init__(self, vel_folder: str, label_folder: str, into_ram: bool = False):
        """Init the velodyne dataset for loading kitti data.

        :param vel_folder: String defining the velodyne dataset folder
        :param label_folder: String defining the label dataset
        """
        super(VelTrainDataset, self).__init__()
        self.vel_folder = vel_folder
        self.vel = sorted(os.listdir(vel_folder))
        self.label_folder = label_folder
        self.labels = sorted(os.listdir(label_folder))
        self.into_ram = into_ram
        self.preprocess = Preprocessing(read_config()['pillars'])

        if self.into_ram:
            print("Loading data into ram...")
            self.vel_ram = []
            self.label_ram = []
            self.load_files_into_ram()
            print("Finished data lodaing!")

    def __len__(self):
        """Returns length of the dataset

        :returns: int defining the length of the dataset
        """
        return len(self.vel)

    def __getitem__(self, idx: int):
        """Returns a single item. Gives data as well as the corresponding kitti label."""
        if idx >= len(self.vel):
            raise ValueError("Index out of bounds.")

        if self.into_ram:
            pcloud = self.vel_ram[idx]
            pcloud = pcloud.reshape([-1,4])
            pil, ind = self.preprocess(pcloud)
            pil = torch.from_numpy(pil)
            ind = torch.from_numpy(ind)

            label = self.label_ram[idx]
        else:
            pil, ind = self.load_data_file(idx)
            label = self.load_label_file(idx)

        return pil, ind, label

    def load_files_into_ram(self):
        for i, file in enumerate(self.vel):
            img_loc = os.path.join(self.vel_folder, file)
            pcloud = np.fromfile(img_loc, dtype=np.float32, count=-1)
            self.vel_ram.append(pcloud)

        for i, file in enumerate(self.labels):
            self.label_ram.append(self.load_label_file(i))

    def load_data_file(self, idx):
        """Load """
        img_loc = os.path.join(self.vel_folder, self.vel[idx])
        pcloud = np.fromfile(img_loc, dtype=np.float32, count=-1)
        pcloud = pcloud.reshape([-1,4])
        """pcloud = torch.as_tensor(pcloud, device="cuda:0")
        pcloud = pcloud.reshape([-1,4])

        pil, ind = calculate_pillars(pcloud)"""
        pil, ind = self.preprocess(pcloud)
        pil = torch.from_numpy(pil)
        ind = torch.from_numpy(ind)
        return pil, ind

    def load_label_file(self, idx):
        """Load the kitti label files from folder given on class init"""
        label_loc = os.path.join(self.label_folder, self.labels[idx])

        label = []
        with open(label_loc, 'r') as f:
            for line in f:
                line = line[:-1].split(" ")
                if line[0] == 'Car':
                    curr_label = line[11:14] + line[8:11] + [line[14]]
                    label.append(curr_label)

        if not label:
            return torch.FloatTensor([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

        # convert str into float
        label = [[float(x) for x in obj] for obj in label]
        return torch.FloatTensor(label)


In [ ]:
#export
class CustomBatch:
    def __init__(self, batch):
        batch_list = list(zip(*batch))
        self.pil_batch = torch.stack(batch_list[0], 0)
        self.ind_batch = torch.stack(batch_list[1], 0)

        self.label_batch_tuple = batch_list[2]
        self.label_mask = []

        label_size = []
        for label in self.label_batch_tuple:
            label_size.append(label.shape[0])

        label_max = max(label_size)
        self.label_batch = []
        for i, label in enumerate(self.label_batch_tuple):
            # label_mask
            mask_ones = torch.ones((label.shape[0]))
            mask_zeros = torch.zeros(label_max - label.shape[0])
            mask = torch.cat((mask_ones, mask_zeros))

            # functional padding only works on 4d tensors
            label = label.unsqueeze(0).unsqueeze(0)
            label = torch.nn.functional.pad(label, (0,0,0,label_max - label.shape[2]))
            self.label_batch.append(label[0,0])
            self.label_mask.append(mask)

        self.label_batch = torch.stack(self.label_batch, dim=0)
        self.label_mask = torch.stack(self.label_mask, dim=0)
        self.label_batch = conv_cam_to_vel_coord(self.label_batch)

    def __iter__(self):
        return iter((self.pil_batch, self.ind_batch, self.label_batch, self.label_mask))

    def pin_memory(self):
        self.pil_batch = self.pil_batch.pin_memory()
        self.ind_batch = self.ind_batch.pin_memory()
        self.label_batch = self.label_batch.pin_memory()
        self.label_mask = self.label_mask.pin_memory()
        return self

In [ ]:
#export
def collate_wrapper(batch):
    return CustomBatch(batch)

In [ ]:
#export
def collate_fn(batch):
    """Returns the correct labeling batch with pil_batch and ind_batch"""
    pil_batch, ind_batch, label_batch, label_size, label_mask = [], [], [], [], []

    transposed_batch = list(zip(*batch))


    for i, data in enumerate(batch):
        # data: [pil, ind, label] for one pcloud
        pil_batch.append(data[0])
        ind_batch.append(data[1])

        label_batch.append(data[2])
        # for figuring out the max label size
        label_size.append(data[2].shape[0])

    pil_batch = torch.stack(pil_batch, dim=0)
    ind_batch = torch.stack(ind_batch, dim=0)

    # Label config
    label_max = max(label_size)

    for i, label in enumerate(label_batch):
        # label_mask
        mask_ones = torch.ones((label.shape[0]))
        mask_zeros = torch.zeros(label_max - label.shape[0])
        mask = torch.cat((mask_ones, mask_zeros))

        # functional padding only works on 4d tensors
        label = label.unsqueeze(0).unsqueeze(0)
        label = torch.nn.functional.pad(label, (0,0,0,label_max - label.shape[2]))
        label_batch[i] = label[0,0]

        label_mask.append(mask)

    label_batch = torch.stack(label_batch, dim=0)
    label_mask = torch.stack(label_mask, dim=0)

    label_batch = conv_cam_to_vel_coord(label_batch)

    return pil_batch, ind_batch, label_batch, label_mask

In [ ]:
#export
class OverfitSampler(torch.utils.data.Sampler):
    def __init__(self,
                 dataset: torch.utils.data.Dataset,
                 batch_size: int,
                 nb_samples: int = 50,
                 shuffle: bool = True):
        self.batch_size = batch_size
        self.nb_samples = nb_samples
        self.shuffle = shuffle

        poss_indices = range(len(dataset))
        self.indices = random.sample(poss_indices, nb_samples)

    def __iter__(self):
        if self.shuffle:
            random.shuffle(self.indices)
        return iter(self.indices)

    def __len__(self):
        return len(self.indices)

    def chunk(self, indices: list, chunk_size: int):
        return torch.split(torch.tensor(indices), chunk_size)


In [ ]:
#export
def conv_cam_to_vel_coord(label_batch: torch.Tensor):
    xc, yc, zc, box_dims = label_batch[:,:,0], label_batch[:,:,1], label_batch[:,:,2], label_batch[:,:,3:]

    xv = zc
    yv = -1 * xc
    zv = -1 * yc

    return torch.cat((xv.unsqueeze(2), yv.unsqueeze(2), zv.unsqueeze(2), box_dims), dim=2)

In [5]:
dt = torch.float32
dev= "cuda:0"
data_path = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training"
label_path = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/label_2/training"
ds = VelTrainDataset(data_path, label_path)
dl = torch.utils.data.DataLoader(ds, batch_size=8, num_workers=0, collate_fn=collate_fn)

for batch in dl:
    print(batch)
    print(batch[2].shape)
    print(batch[3].shape)
    break


(tensor([[[[ 6.0830e+00,  6.1060e+00,  6.1110e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 8.5110e+00,  8.5710e+00,  8.5710e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 1.5560e+00,  1.5400e+00,  1.4960e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],

         [[-8.8730e+00, -8.8040e+00, -8.9140e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-1.8960e+00, -1.8140e+00, -1.8240e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-3.8640e+00, -3.9760e+00, -3.8420e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          ...,
    

In [6]:
arr1 = torch.rand((2, 7))
arr1 = arr1.reshape(1, 1, 2, 7)
arr2 = torch.nn.functional.pad(arr1, (0,0,0,1), mode='reflect')
arr2 = arr2[0,0]



arr1, arr1.shape, arr2, arr2.shape


(tensor([[[[0.7715, 0.0083, 0.8006, 0.0177, 0.6741, 0.1756, 0.2301],
           [0.8170, 0.6701, 0.8676, 0.5520, 0.5381, 0.2167, 0.2795]]]]),
 torch.Size([1, 1, 2, 7]),
 tensor([[0.7715, 0.0083, 0.8006, 0.0177, 0.6741, 0.1756, 0.2301],
         [0.8170, 0.6701, 0.8676, 0.5520, 0.5381, 0.2167, 0.2795],
         [0.7715, 0.0083, 0.8006, 0.0177, 0.6741, 0.1756, 0.2301]]),
 torch.Size([3, 7]))